In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt

In [25]:
df = pd.DataFrame()
for year in [23,24]:
    url = f'https://stats.nba.com/stats/leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&ISTRound=&LeagueID=00&PlayerOrTeam=T&Season=20{year-1}-{year}&SeasonType=Regular%20Season&Sorter=DATE'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64)', 'x-nba-stats-origin': 'stats', 'x-nba-stats-token': 'true', 'Host':'stats.nba.com', 'Referer':f'https://www.nba.com/'}
    r= requests.get(url, headers=headers, timeout = 7)
    data = json.loads(r.text)
    games = data['resultSets'][0]['rowSet']
    ids = [[info[4],info[6].split(' @ ')[0],info[6].split(' @ ')[1]] for info in games if '@' in info[6]]
    ids = sorted(ids)
    print(year,len(ids))
    temp_df = pd.DataFrame.from_dict(ids)
    temp_df.columns=[['game_id','home','away']]
    temp_df['season'] = year
    if df.empty:
        df = temp_df.copy(deep=True)
    else:
        df = pd.concat((df,temp_df))
df

23 1230
24 71


,game_id,home,away,season
0,0022200001,PHI,BOS,23
1,0022200002,LAL,GSW,23
2,0022200003,ORL,DET,23
3,0022200004,WAS,IND,23
4,0022200005,HOU,ATL,23
...,...,...,...,...
66,0022300127,LAC,LAL,24
67,0022300128,TOR,PHI,24
68,0022300129,DET,NOP,24
69,0022300130,ORL,UTA,24


In [26]:
df.to_csv('game_ids.csv',index=False)